<a href="https://colab.research.google.com/github/yt-oh96/Lab_tutorial/blob/master/Task1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import cv2
import numpy as np
#from PIL import Image
import matplotlib.pyplot as plt

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
ori_image = cv2.imread('image1.tif',1)
dis_image = cv2.imread('image2.tif',1)

plt.imshow(np.hstack((ori_image, dis_image)))

In [0]:
fig, axes = plt.subplots(1,2)
axes[0].hist(ori_image.ravel(), bins=30)
axes[1].hist(dis_image.ravel(), bins=30)

In [0]:
def optim(ori, dis, mode='', distortion=''):
  x,y = ori.shape[:2]
  
  error = []
  if distortion == 'translation':
    i=0
    for dx in range(-x,x):
      i+=1
      error_y=[]
      j=0
      for dy in range(-y,y):
        
        j+=1
        M = np.float32([[1,0,dx],[0,1,dy]])
        dst = cv2.warpAffine(ori, M, (x,y))

        if mode == 'ssd':
          err = SSD(dst, dis)
        elif mode == 'mi':
          err = -MI(dst,dis)
        
        error_y.append(err)
        
        print(str(i)+','+str(j)+'/512'+', error : ',err)
      error.append(error_y)

  elif distortion == 'rotation':
    w,h = ori.shape[:2]
    for i in range(360):
      
      rotation_mat = cv2.getRotationMatrix2D((w/2,h/2),i,1)
      rotation_image = cv2.warpAffine(ori, rotation_mat, (w,h))
       
      if mode == 'ssd':
        err = SSD(rotation_image, dis)
      elif mode == 'mi':
        err = -MI(rotation_image,dis)

      print(str(i)+','+'/360'+', error : ',err)
      error.append(err)  
  return error


In [0]:
def entropy(image):
  image = np.asarray(image)

  p_image = image / image.sum()

  p_image = p_image[np.nonzero(p_image)]
  return -np.sum(p_image*np.log2(p_image))

def MI(image_A, image_B):
  image_A = np.asarray(image_A.ravel())
  image_B = np.asarray(image_B.ravel())
  print(image_A.shape)

  histogram_A = np.histogram(image_A, 30)[0]
  histogram_B = np.histogram(image_B, 30)[0]
  histogram_AB = np.histogram2d(image_A, image_B, 30)[0]

  H_a = entropy(image_A)
  H_b = entropy(image_B)
  H_ab = entropy(histogram_AB)
  #H_ab = entropy(np.concatenate([image_A, image_B]))
  print(H_a, ' ',H_b,' ', H_ab)
  MI_ab = H_a + H_b - H_ab

  return MI_ab

mi = MI(ori_image, dis_image)
print(mi)

In [0]:
final_mi = optim(ori_image, dis_image, mode='mi')

In [0]:
import pandas as pd

pd_final = pd.DataFrame(final_mi)
Z_mi = pd_final.fillna(-15)
Z_mi = np.asarray(Z_mi)

Z_mi_min = np.min(Z_mi, axis=0)
Z_mi_min_idx = np.argmin(Z_mi_min)

Z_mi_min_y = np.min(Z_mi, axis=1)
Z_mi_min_idx_y = np.argmin(Z_mi_min_y)

Z_mi_min_idx_y -= 256
Z_mi_min_idx -= 256

print('min_idx = ',Z_mi_min_idx_y,',',Z_mi_min_idx)

In [0]:
from mpl_toolkits.mplot3d import Axes3D
x = np.linspace(-256,255,512)
y = np.linspace(-256,255,512)
Z_np = np.array(Z_mi)
xx, yy = np.meshgrid(x,y)

plt.figure(figsize=(5,5))
ax = plt.subplot(1,1,1,projection='3d')
ax.plot_surface(xx,yy,Z_np)
ax.view_init(30,30)
plt.show()

In [0]:
hist_2d, x_edges, y_edges = np.histogram2d(ori_image.ravel(), dis_image.ravel(), bins=50)
plt.imshow(hist_2d.T, origin='lower')

In [0]:
M = np.float32([[1,0,14],[0,1,9]])
dst_image = cv2.warpAffine(ori_image, M, ori_image.shape[:2])

hist_2d, x_edges, y_edges = np.histogram2d(dst_image.ravel(), dis_image.ravel(), bins=50)
plt.imshow(hist_2d.T, origin='lower')

In [0]:
plt.subplot(121)
plt.imshow(dst_image, cmap='gray')
plt.title('ori_image' )

plt.subplot(122)
plt.imshow(dis_image, cmap='gray')
plt.title('dis_image')

plt.show()